In [ ]:
# IMPORTING python libraries
import numpy as np
import imageio
from tqdm import tqdm
import cv2
import pandas as pd
from sklearn import preprocessing as prep
from google.colab import drive
import tensorflow as tf

In [ ]:
class ETL:

  def __init__(self):

    # This method uses Google Drive and zipped file. Please change the links as per your convenience

    drive.mount('/content/drive')
    !unzip -uq "/content/drive/My Drive/Colab Notebooks/TopOPT/data/image.zip"

    self.img_path = "data/"
    self.csv_path =  "/content/drive/My Drive/Colab Notebooks/TopOPT/data/"
    self.gsize = (7,7)
    self.IMG_SHAPE = (32,64)
    self.no_of_data_points = 15000
    self.csv_splits = [[1,1500],[1501,2200],[2201,2700],[2700,4500],[4501,5225],[5225,7000],[7001,8000],
                       [8000,10000],[10000,11725],[11725,12500],[12500,15000]]


  def initiate(self,augment = False):
    
    self.load_dataset()
    self.clean_data()

    if augment is True:
      self.augment_data()
      print('Augmented')
    
    self.data_norm()


  # Load the dataset-----------------------
  def load_dataset(self):
    
    # Loading Time and compliance data
    self.load_comp_time() 
  
    # Loading images as 6 channels
    self.train_images = []

    for i in tqdm(range(1,self.no_of_data_points+1)):
      i_mod = "{:01d}".format(i)  
      # try:   
      self.train_images.append(self.load_image(i_mod))
      #except:
       # print('Not found {}', i) 

  # Cleaning DataSet---------------------------- 
  def clean_data(self):
    faulty_cases = self.find_faulty() # FINDING FAULTY CASES
    self.comp_time = np.delete(self.comp_time,faulty_cases,axis=0) # DELETING FAULTY CASES from COMP_TIME
    self.comp_time = self.comp_time[:,[1,2]] # REMOVING EXTRA ROWS
    self.train_images = np.delete(self.train_images,faulty_cases,0) # DELETING FAULTY CASES FROM TRAIN IMAGES

  # Find Faulty Cases---------------------
  def find_faulty(self):
    data = self.comp_time
    # Due to automatic generator for data, some generated cases resulted in an invalid solution, need to clean data

    # 0 --> INDEX, 1--> Compliance, 2--> Compute Time
    # Imaginary Numbers generated, additional rows generated 3,4,5
    
    imag_faulty = data[:,4]  # IMAGINARY NUMBER IN LAST ROWS
    K1 = np.nonzero(imag_faulty>0)

    comp_faulty = data[:,1] # COMPLIANCE TOO HIGH --> NOT OPTIMIZED BY THE SIMP CORRECTLY 

    cmf = comp_faulty[np.nonzero(comp_faulty>0)]
    cmf = cmf[np.nonzero(cmf<5000)]
    ts = np.mean(cmf) + 2*np.std(cmf)
    K2 = np.nonzero(comp_faulty>ts) # Compliance is not in 2 std of mean  

    return np.union1d(K1,K2)   # Return faulty cases
    
    # COMP TIME----------------------------------------
  def load_comp_time(self):

        for s in self.csv_splits:
          a = s[0]
          b = s[1]
          print(a, b)
          # try:
          if a is 1:
            self.comp_time = self.load_csv(a,b)
          else:
            self.comp_time = np.vstack((self.comp_time,self.load_csv(a,b)))
          # except:
          #    print('Not found {} - {}',a,b)
        self.comp_time = np.array(self.comp_time)

        
  def load_csv(self,a,b):
        a = "{:01d}".format(a) 
        b = "{:01d}".format(b) 

        path = self.csv_path + "comp_time_" + a + "_" + b + "_.csv"
        CT = pd.read_csv(path, header= None)

        # Complex to real     
        comp_time = self.complex_to_real(CT,int(b))
        comp_time = comp_time[int(a):int(b),:]
        return comp_time
        

  def complex_to_real(self,data,ds):
        # 0 --> INDEX + j compliance, 1--> compute_time + j X, 2--> Y + Z
        # OUTPUT DATA --> 0--> index, 1-->compliance, 2-->time, 3-->0ss 4--->ERROR 5-->0s

        comp_time = np.zeros((ds,6), dtype = float)

        for i in range(ds):
          for j in range(3):

            a = data[j][i]  
            real, imag = a.split('+') # SPLITTING COMPLEX to real
            imag = imag[:-1] # Mag of imaginary number
            real = float(real) # Float
            imag = float(imag)
            comp_time[i,2*j] = real
            comp_time[i,2*j+1] = imag

        return comp_time

    # LOAD IMAGE--------------------------------------
  def load_image(self,i_mod,blur=True):
        path = self.img_path
        
        OUT = 255-self.image_read(path + "OUT/"+"OUT_"+ i_mod + ".png") # 1 represents presence of material
        BX = 255-self.image_read(path + "B/"+"BFX_"+ i_mod + ".png") # 1 represents fixeddofs
        BY  = 255-self.image_read(path + "B/"+"BFY_"+ i_mod + ".png") # 1 represents fixeddofs
        FX  = self.image_read(path + "F/"+"FX_"+ i_mod + ".png") # 0.5 represents 0 force
        FY = self.image_read(path + "F/"+"FY_"+ i_mod + ".png")  # 0.5 represents 0 force 
        VF  = self.image_read(path + "VF/"+"VF_"+ i_mod+".png")

        if blur == True:
          BX  =  cv2.GaussianBlur(BX,self.gsize,cv2.BORDER_DEFAULT)
          BY =  cv2.GaussianBlur(BY,self.gsize,cv2.BORDER_DEFAULT)
          FX  =  cv2.GaussianBlur(FX,self.gsize,cv2.BORDER_DEFAULT)
          FY =  cv2.GaussianBlur(FY,self.gsize,cv2.BORDER_DEFAULT)  
        
        # Default arrangement of channels
        IMG = np.stack([OUT,BX,BY,FX,FY,VF],2)
        return IMG


  def image_read(self,dir,a = False):

    if a is False:
      img = cv2.imread(dir,0)
    else:
      img = cv2.imread(dir)

    width = self.IMG_SHAPE[1]
    height = self.IMG_SHAPE[0]
    dim = (width, height)
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    resized =  np.asarray(resized).astype('float32')
    return resized

  # AUGMENTING DATA---------------------
  def augment_data(self):
      images = []
      comp = []

      for img,c in tqdm(zip(self.train_images,self.comp_time)):

        flipped = np.zeros_like(img)
        updown = np.zeros_like(img)
        
        for i in range(6):
          s = img[:,:,i:i+1]
          flipped[:,:,i:i+1] = tf.image.flip_left_right(s)
          updown[:,:,i:i+1] = tf.image.flip_up_down(s) 

        images.append(img)  
        comp.append(c)
        images.append(flipped)
        comp.append(c)
        images.append(updown)
        comp.append(c)

      self.train_images = np.array(images).astype('float32')
      self.comp_time = np.array(comp).astype('float32')  
    
  def data_norm(self):

      self.train_images = np.array(self.train_images, dtype = float)
      self.train_images = (self.train_images - 127.5)/127.5

      # self.comp_time = np.array(self.comp_time, dtype = float)
      # self.comp_time = prep


In [ ]:
dataset = ETL()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset.initiate(augment=True)

1 1500
1501 2200
2201 2700
2700 4500
4501 5225
5225 7000
7001 8000
8000 10000
10000 11725
11725 12500
12500 15000


100%|██████████| 15000/15000 [00:17<00:00, 854.62it/s]
14116it [00:50, 280.44it/s]


Augmented


In [ ]:
dataset.train_images.shape

(42348, 32, 64, 6)

In [ ]:
dataset.comp_time.shape

(42348, 2)